In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#checking the datas valuable description
pd.read_csv("VariableDefinitions (1).csv")

,Variable Definitions,Unnamed: 1
0,country,Country interviewee is in.
1,year,Year survey was done in.
2,uniqueid,Unique identifier for each interviewee
3,location_type,"Type of location: Rural, Urban"
4,cellphone_access,"If interviewee has access to a cellphone: Yes, No"
5,household_size,Number of people living in one house
6,age_of_respondent,The age of the interviewee
7,gender_of_respondent,"Gender of interviewee: Male, Female"
8,relationship_with_head,The interviewee’s relationship with the head o...
9,marital_status,The martial status of the interviewee: Married...


In [3]:
#examinining the train dataset
df_train=pd.read_csv("train.csv")
df_train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [4]:
d


TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [ ]:
df_train["marital_status"].value_counts()

In [ ]:
#examining the test dataset
df_test=pd.read_csv("test.csv")
df_test.head()

In [ ]:
#getting the  shape of train  and test datsets
print(df_train.shape)
print(df_test.shape)

In [ ]:
#submission files
submission=pd.read_csv("SampleSubmission.csv")
submission

In [ ]:
#descriptive discription about the  test dataset of numeric values of test_data
df_train.describe()

In [ ]:
df_train.info()

hhalf of the people have 35yrs and below while 75% and below  have 49 yrs and below

In [ ]:
#checking about the categprical distribution
df_train.describe(include="object")

In [ ]:
#checking whether the dataset is balanced
df_train["bank_account"].value_counts()


imbalanced dataset

In [ ]:
#getting to know about datasets
df_train.nunique()

In [ ]:
#checking some visiualization for normal disribution 
df_train.hist(bins=25,figsize=(10,8))
plt.show()

In [ ]:
#checkinmg for normal distibution by use of kurtosis
df_train.kurtosis()

the data  features shows  slightly  normal distibution

In [ ]:
#by use of boxplots
sns.boxplot(data=df_train, orient="h")

In [ ]:
sns.countplot(data=df_train,orient="h")

In [ ]:
df_train.select_dtypes(include="O")

In [ ]:
#importing somme libraries for preeprocessing
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
oh=OneHotEncoder()
from sklearn.preprocessing import StandardScaler
std=StandardScaler()

In [ ]:
df_train.head()

In [ ]:
df_train.head()
df_train.shape


In [ ]:
#dividing the train data to depedent and indepedent variables
#transforming the target variable
df_train["bank_account"]=le.fit_transform(df_train["bank_account"])
y_train=df_train["bank_account"]
x_train=df_train.drop("bank_account",axis=1,inplace=True)


In [ ]:
def preprocsessing_data(df_train):
    # categorical features to be onverted to One Hot Encoding
    ohe_df=pd.DataFrame(df_train, columns=["relationship_with_head","country","location_type"])
    # generate binary values using get_dummies
    dum_df = pd.get_dummies(ohe_df, columns=["relationship_with_head","country","location_type"], prefix=["Type_is","Type_is","Type_is"])
    # merge with main df bridge_df on key values
    df_train=df_train.join(dum_df)
    
    # Label Encoder conversion
    df_train["cellphone_access"] = le.fit_transform(df_train["cellphone_access"])
    df_train["gender_of_respondent"] = le.fit_transform(df_train["gender_of_respondent"])
    df_train["year"] = le.fit_transform(df_train["year"])
    df_train["marital_status"] = le.fit_transform(df_train["marital_status"])
    df_train["education_level"] = le.fit_transform(df_train["education_level"])
    df_train["job_type"] = le.fit_transform(df_train["job_type"])

    
    # drop uniquid column
    df_train=df_train.drop(columns=["uniqueid","relationship_with_head","country","location_type"],axis=1)
    
    # scale our data into range of 0 and 1
    scaler=StandardScaler() 
    df_train=scaler.fit_transform(df_train)
    
    return df_train
    

In [ ]:
procesed_train=preprocsessing_data(df_train)
processd_test=preprocsessing_data(df_test)

In [ ]:
print(procesed_train.shape)
print(processd_test.shape)

In [ ]:
#spltting train data to test and validation dataset
from sklearn.model_selection import train_test_split
x_Train,x_valid,y_Train,y_valid=train_test_split(procesed_train,y_train,test_size=0.1,
                                                 stratify = y_train,
                                                 random_state=42)

In [ ]:
#model building
#importing models from various libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier    
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
#builiding pippeline for the models
pipe_dt=Pipeline([("decision tree",DecisionTreeClassifier())])

pipe_rf=Pipeline([("random forest",RandomForestClassifier())])

pipe_svc=Pipeline([("svc",SVC())])

pipe_knearest=Pipeline([("kneigbour",KNeighborsClassifier())])

pipe_naibayes=Pipeline([("gaussian nb",GaussianNB())])

pipe_adboost=Pipeline([("adboost",AdaBoostClassifier())])

pipe_gradientboost=Pipeline([("gradient",GradientBoostingClassifier())])

pipe_extratree=Pipeline([("etra_tree",ExtraTreesClassifier())])

In [ ]:
#forming a list for the modelels
pipe_models=[pipe_dt,pipe_rf,pipe_svc,pipe_knearest,pipe_naibayes,pipe_adboost,pipe_gradientboost,pipe_extratree]
pipe_models

In [ ]:
pipe_dict={0:"DecisionTreeClassifier",
         1:"RandomForestClassifier",
         2:"SVC",
         3:"KNeighborsClassifier",
         4:"GaussianNB",
         5:"AdaBoostClassifier",
         6:"GradientBoostingClassifier",
         7:"ExtraTreesClassifier"}

In [ ]:

#training the models
for model in pipe_models:
    model.fit(x_Train,y_Train)

In [ ]:
#the scores of the models
for i,model in enumerate(pipe_models):
    print("{} valid score is {}".format(pipe_dict[i],model.score(x_valid,y_valid)))

AdaBoostClassifier valid score and the GradientBoostingClassifier valid score  classifier gives the highest score and therefore we are going to hypertune them

In [ ]:
#using grid search
adbost=AdaBoostClassifier()
param_grid={"n_estimators":[50,100,200,250,300,400,450,500],
            "learning_rate":[1.0,0.1,0.01]}
ad_bostgrid= GridSearchCV(adbost,param_grid,cv=5,n_jobs=1)

ad_bostgrid.fit(x_Train,y_Train)


In [ ]:
y_predict=ad_bostgrid.predict(x_valid)
print(confusion_matrix(y_valid,y_predict))
print("accuracy_score is",accuracy_score(y_valid,y_predict))
print("error is",1-accuracy_score(y_valid,y_predict))

In [ ]:
gradient=GradientBoostingClassifier(n_estimators=200,max_depth=4,min_samples_split=2,learning_rate=0.1, min_samples_leaf=2)
gradient.fit(x_Train,y_Train)

In [ ]:
y_predict=gradient.predict(x_valid)
print(confusion_matrix(y_valid,y_predict))
print("accuracy_score is",accuracy_score(y_valid,y_predict))
print("error is",1-accuracy_score(y_valid,y_predict))

after hypertuning the model accuracy increased

In [ ]:
prediction=gradient.predict(processd_test)
prediction

In [ ]:
#submission file
submission={"unique_id":df_test["uniqueid"],"bank_account":prediction}
submission_df=pd.DataFrame(submission)
submission_df

In [ ]:
#Create submission csv file csv file
submission_df.to_csv('first_submission.csv', index = False)